# Type Inference by hand

Using a subset of the Lettuce grammar and functions from the TypeInferenceLettuce reading, we gain an intuition for the mechanics of type inference by manually solving simple examples.


## Lettuce Grammar

We will use the following grammar that does not include recursion. Note that we do not allow type annotations in our version.

$$\begin{array}{rcll}
\mathbf{Program} & \rightarrow & TopLevel(\mathbf{Expr}) \\[5pt]
\mathbf{Expr} & \rightarrow & Const(\mathbf{Number}) \\
 & | & Ident(\mathbf{Identifier}) \\
 & | & Plus(\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Mult(\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Eq(\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Geq (\mathbf{Expr}, \mathbf{Expr}) \\
 & | & IfThenElse(\mathbf{Expr}, \mathbf{Expr}, \mathbf{Expr}) \\
 & | & Let( \mathbf{Identifier}, \mathbf{Expr}, \mathbf{Expr}) \\
 & | & FunDef( \mathbf{Identifier}, \mathbf{Expr}) \\ 
 & | & FunCall(\mathbf{Expr}, \mathbf{Expr})\\
\end{array}$$


In [1]:
sealed trait Expr

case class Const(f: Double) extends Expr
case class Ident(x: String) extends Expr
case class Plus(e1: Expr, e2: Expr) extends Expr
case class Geq (e1: Expr, e2: Expr) extends Expr
case class And(e1: Expr, e2: Expr) extends Expr
case class Not(e: Expr) extends Expr
case class IfThenElse(e: Expr, e1: Expr, e2: Expr) extends Expr
case class Let(x: String, e1: Expr, e2: Expr) extends Expr
case class FunDef(param: String, body: Expr) extends Expr
case class FunCall(e1: Expr, e2: Expr) extends Expr

defined trait Expr
defined class Const
defined class Ident
defined class Plus
defined class Geq
defined class And
defined class Not
defined class IfThenElse
defined class Let
defined class FunDef
defined class FunCall

## Abstract Syntax Tree for Types

In lecture, we learned to infer types by performing two steps:

1. Generate a system of equations: these are our type constraints
2. Solve the system of equations using the unification approach.

Let us define an AST for our types along with some utilities.

In [2]:
sealed trait Type
case object NumType extends Type
case object BoolType extends Type
case class FunType(type1: Type, type2: Type) extends Type
case class TypeVar(name:String) extends Type

type TypeEnvironment = Map[String, Type]
type ListOfEquations = List[(Type, Type)]

object TypeVarGenerator {
    var counter = 0
    
    def getFreshTypeVariable(id: String): TypeVar = {
        val t = TypeVar("type_" + id + "_" + counter.toString)
        counter = counter + 1
        t
    }
    
    def resetCounter = {
        counter = 0
    }
}

defined trait Type
defined object NumType
defined object BoolType
defined class FunType
defined class TypeVar
defined type TypeEnvironment
defined type ListOfEquations
defined object TypeVarGenerator

## Generating Type Variables and Constraints

The following rules allow us to generate constraints:
    - `Const(_)`: 
        - return NumType, Nil
    - `Ident(s)`: 
        - look up s in our type environment and throw an error if it is not found
        - return type(s), Nil
    - `Plus(e1, e2)`:   
        - generate constraints for `e1` and add a constraint: `type(e1) == NumType`
        - generate constraints for `e2` and add a constraint: `type(e2) == NumType`
        - return NumType, constraints
    - `Geq(e1, e2)`:  
        - generate constraints for `e1` and add a constraint: `type(e1) == NumType`
        - generate constraints for `e2` and add a constraint: `type(e2) == NumType`
        - return BoolType, constraints
    - `And(e1, e2)`:  
        - generate constraints for `e1` and add a constraint: `type(e1) == BoolType`
        - generate constraints for `e2` and add a constraint: `type(e2) == BoolType`
        - return BoolType, constraints
    - `Not(e1)`:  
        - generate constraints for `e1` and add a constraint: `type(e1) == BoolType`
        - return BoolType, constraints
    - `IfThenElse(e, e1, e2)`:
        - generate constraints for `e` and add a constraint: `type(e) == BoolType`
        - generate constraints for `e1` and `e2` and add a constraint: `type(e1) == type(e2)`
        - return `type(e1)`, constraints
    - `Let(x, e1, e2)`:
        - create a new `TypeVar` for the argument x and call it `type_x_0`
        - generate constraints for `e1` and create a new type environment augmented with `x -> type_x_0`
        - generate constraints for `e2` using the new type environment
        - add the constraint `type_x_0 == type(e1)` 
        - return `type(e2)`, constraints
    - `FunDef(x, e1)`:
        - create a new `TypeVar` for the argument x and call it `type_x_0`
        - create a new type environment augmented with `x -> type_x_0`
        - generate constraints for `e1` using the new type environment 
        - return `FunType(type_x_0, type(e1))`, constraints
    - `FunCall(e1, e2)`:
        - create a new `TypeVar` for the function call and call it `type_fcall_0`
        - generate constraints for `e1` and `e2`
        - add the constraint `type(e1) == FunType(type(e2), type_fcall_0)`
        - return `type_fcall_0`, constraints

From the readings, we have code that will do this automatically. We will go through some examples and build up the constraints manually, then compare the results to the output of `generateEquations`.


In [3]:
case class ErrorException(s: String) extends Exception 

def generateEquations(e: Expr, alpha: TypeEnvironment): (Type, ListOfEquations) = 
    e match {
        case Const(f) => (NumType, Nil) // If expr is a constant, return num and Nil
        case Ident(id) => {
              if (alpha contains id){
                  (alpha(id), Nil)
              } else {
                  throw new ErrorException(s"Used undeclared identifier $id -- type error")
              }
        }

        case Plus(e1, e2) => {
             val (t1, lst1) = generateEquations(e1, alpha) // Gen. eqs for e1
             val (t2, lst2) = generateEquations(e2, alpha) // Gen. eqs for e2
             val combinedList = lst1 ++ lst2 ++ List( (t1, NumType), (t2, NumType) )
             (NumType, combinedList) // The overall type of Plus is a num
        }

        case Geq(e1, e2) => {
             val (t1, lst1) = generateEquations(e1, alpha) // Gen. eqs for e1
             val (t2, lst2) = generateEquations(e2, alpha) // Gen. eqs for e2
             val combinedList = lst1 ++ lst2 ++ List( (t1, NumType), (t2, NumType) )
             (BoolType, combinedList) // Overall type of Geq is a boolean
        }

        case And(e1, e2) => {
            val (t1, lst1) = generateEquations(e1, alpha) // Gen. eqs for e1
            val (t2, lst2) = generateEquations(e2, alpha) // Gen. eqs for e2
            val combinedList = lst1 ++ lst2 ++ List( (t1, BoolType), (t2, BoolType) )
            (BoolType, combinedList) // Overall type of And is a boolean
        }

        case Not(e1) => {
            val (t1, lst1) = generateEquations(e1, alpha) // Gen. eqs for e1
            val combinedList = lst1 ++ List( (t1, BoolType) )
            (BoolType, combinedList) // Overall type of And is a boolean
        }

        case IfThenElse(e, e1, e2) => {
             val (t0, lst0) = generateEquations(e, alpha) // Gen. eqs for e
             val (t1, lst1) = generateEquations(e1, alpha) // Gen. eqs for e1
             val (t2, lst2) = generateEquations(e2, alpha) // Gen. eqs for e2
             val combinedList = lst1 ++ lst2 ++ List( (t0, BoolType), (t1, t2) )
             (t1, combinedList) // Overall type of IfThenElse is t1
        }

        case Let(x, e1, e2) => {
           val tx = TypeVarGenerator.getFreshTypeVariable(x)
           val (te1, listE1) = generateEquations(e1, alpha)
           val newAlpha = alpha + (x -> tx)
           val (te2, listE2) = generateEquations(e2, newAlpha)
           val combinedList = listE1 ++ listE2 ++ List( (tx, te1) )
           (te2, combinedList)
        }  

        case FunDef(param, body) => {
            val tparam = TypeVarGenerator.getFreshTypeVariable(param) // Gen. fresh type variable 
            val newEnv = alpha + (param -> tparam)
            val (tbody, listBody) = generateEquations(body, newEnv)
            val fnType = FunType(tparam, tbody)
            (fnType, listBody)
        }

        case FunCall(e1, e2) => {
            val (te1, listE1) = generateEquations(e1, alpha)
            val (te2, listE2) = generateEquations(e2, alpha)
            val te = TypeVarGenerator.getFreshTypeVariable("fcall")
            val newTypeConstraint = (te1, FunType(te2, te) ) // te1 == te2 => te
            val combinedList = listE1 ++ listE2 ++ List(newTypeConstraint)
            (te, combinedList)
        }  
    }

defined class ErrorException
defined function generateEquations

In [4]:
def typeToString(t: Type) : String = t match {
    case NumType => "NumType"
    case BoolType => "BoolType"
    case FunType(t1, t2) => "FunType("+(typeToString(t1)) + " => " + (typeToString(t2)) +")"
    case TypeVar(str) => str
}

def prettyPrintTypeEqs (lst: List[(Type, Type)]): Unit = {
    lst.foreach {
        case (t1, t2) => {
            println(typeToString(t1) + " == " + typeToString(t2))
        }
    }
}

defined function typeToString
defined function prettyPrintTypeEqs

## Examples


In [5]:
// let x = 2 in x + 1
val expr = Let("x", Const(2), Plus(Ident("x"), Const(1)))

// 1. The top level expression is a Let
//     - `Let(x, e1, e2)`:
//         - create a new `TypeVar` for the argument x and call it `type_x_0`
//         - generate constraints for `e1` and create a new type environment augmented with `x -> type_x_0`
//         - generate constraints for `e2` using the new type environment
//         - add the constraint `type_x_0 == type(e1)` 
//         - return `type(e2)`, constraints

// 2. e1 = Const(2), so type(e1) = NumType and we evaluate e2 using alpha ++ (x -> NumType)

// 3. e2 = Plus(Ident("x"), Const(1))
//     - `Plus(e1_hat, e2_hat)`:   
//         - generate constraints for `e1_hat` and add a constraint: `type(e1_hat) == NumType`
//         - generate constraints for `e2_hat` and add a constraint: `type(e2_hat) == NumType`
//         - return NumType, constraints

// 4. e1_hat = Ident("x") which evaluates to type_x_0, we add our first constraint type_x_0 == NumType.

// 5. e2_hat = Const(1) which evaluates to NumType, we add our second constraint NumType == NumType

// 6. type(e2) = NumType, so add the constraint `type_x_0 == NumType`

// 7. type(e2) = NumType, so we return NumType

val (typ, lstOfEqs) = generateEquations(
    expr,
    Map[String, Type]()
)
TypeVarGenerator.resetCounter

println("-- Generated Eqs --")
prettyPrintTypeEqs(lstOfEqs)
println("The overall program has type: " + typeToString(typ))



-- Generated Eqs --
type_x_0 == NumType
NumType == NumType
type_x_0 == NumType
The overall program has type: NumType


expr: Let = Let("x", Const(2.0), Plus(Ident("x"), Const(1.0)))
typ: Type = NumType
lstOfEqs: ListOfEquations = List(
  (TypeVar("type_x_0"), NumType),
  (NumType, NumType),
  (TypeVar("type_x_0"), NumType)
)

In [16]:
// if 5 > 2 then 3 else function (x) x + 1
val expr = ???

val (typ, lstOfEqs) = generateEquations(
    expr,
    Map[String, Type]()
)
TypeVarGenerator.resetCounter

println("-- Generated Eqs --")
prettyPrintTypeEqs(lstOfEqs)
println("The overall program has type: " + typeToString(typ))



: 

In [17]:
// let f = function (x)  x >= 5 in 
//   f(2)

val expr = ???

val (typ, lstOfEqs) = generateEquations(
    expr,
    Map[String, Type]()
)
TypeVarGenerator.resetCounter

println("-- Generated Eqs --")
prettyPrintTypeEqs(lstOfEqs)
println("The overall program has type: " + typeToString(typ))



: 

## Do all of these examples typecheck? If no, which ones do not and why?
